In [2]:
'''
This program was written by Joe Stephenson during the Sanger Scholar Program. You are free 
to use/alter it however you want. I hope that this helps you collect data for whatever purpose
you need it for!
'''
import os
import pandas as pd
from numpy import array
import numpy as np
import math
import time
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from datetime import datetime
from datetime import date
from collections import defaultdict
from iexfinance.stocks import get_historical_intraday

#example list of companies.
companies = [
        #utlilities 
            'AES', 'LNT', 'AEE', 'AEP', 'AWK',
            'CNP', 'CMS', 'ED', 'D', 'DTE',
            'DUK', 'EIX', 'ETR', 'EVRG', 'ES',
            'EXC', 'FE', 'NEE', 'NI', 'NRG',
            'PNW', 'PPL', 'PEG', 'SRE', 'SO',
            'WEC', 'XEL',
        #energy
            'APC','APA', 'BHGE', 'COG',
            'CVX', 'XEC', 'CXO', 'COP', 'DVN', 
            'EOG', 'EQT', 'XOM', 'HAL', 'HP',
            'HES', 'KMI', 'MRO', 'MPC', 'NOV',
            'NFX', 'NBL', 'OXY', 'OKE', 'PSX',
            'PXD', 'RRC', 'SLB', 'FTI', 'VLO', 'WMB', '']
MEASUREMENTS = [
                'average',
                'changeOverTime',
                'close',
                'date',
                'high',
                'low',
                'marketAverage',
                'marketChangeOverTime',
                'marketClose', 
                'marketHigh',
                'marketLow',
                'marketNotional',
                'marketNumberOfTrades',
                'marketOpen',
                'marketVolume',
                'notional',
                'open',
                'volume'
                        ]
#We want to know how many 
NUM_MEASURE = len(MEASUREMENTS)
#We create a variable to keep track of how many companies we are drawing data on
COUNT = len(companies)
#we need to know the day today
TODAY = date.today()
#we want to create a variable that stores the day furthest in the past that we can draw data on
THEN = TODAY - relativedelta(months=2)
#I think this is the furthest you can draw data on, correct me if i am wrong
THEN = date(THEN.year, THEN.month, 1)
#get the total number of days in between
TOTAL = TODAY - THEN
TOTAL = TOTAL.days
#get columns to print out what variable we can select from.
GET_COLUMNS = get_historical_intraday('LNT', THEN, output_format='pandas') 
#prints out this information
print(THEN, "to", TODAY, 'is the range of data being pulled')
print(TOTAL, "total number of days will be pulled")
print(COUNT, "total number of companies will be pulled from")
print("""
""")
print("A list of all the variables that you can pull data on")
print()
#creates a list of our column names and prints them out
columns = []
for col in GET_COLUMNS.columns: 
    print(col) 
    columns.append(col)

    


68
2019-03-01 to 2019-05-08 is the range of data being pulled
68 total number of days will be pulled
57 total number of companies will be pulled from


A list of all the variables that you can pull data on

average
changeOverTime
close
date
high
label
low
marketAverage
marketChangeOverTime
marketClose
marketHigh
marketLow
marketNotional
marketNumberOfTrades
marketOpen
marketVolume
notional
numberOfTrades
open
volume


In [ ]:
#we define a function, wutfinance() that we can pass our MEASUREMENTS array into
#if you want to change what variables are pulled, add or delete from this list.
def wut_iex(variable):
    #we want to go through all of our companies
    for x in companies[0:-1]:
        #define an empty array to store all of the dates that we get data on
        dates = []
        t_values = []
        #create an empty defaultdict which we will use to create lists of lists
        data_list = defaultdict(list)
        #we have to pull one day at a time with IEX finance. 
        for t in range(TOTAL):
            #we update the day that we are drawing data on
            END_DATE = THEN + timedelta(days=t)
            #print out the date that we are drawing data for
            print('Obtaining data for', x, 'for day', t+1)
            #obtain this data from the IEX finance data api
            data = get_historical_intraday(x, END_DATE, output_format='pandas')
            #if IEX returns a dataset, it will invariably have length of 390.
            #we can take advantage of this and skip days that do not return data
            if len(data) == 390: 
                #we want to keep track of what days we do draw data, this way we can label 
                #our datasets index later
                dates.append(END_DATE)
                #We create a unique lists for each variable. This will allow us to store
                #single variables on each company.
                for okay in MEASUREMENTS:
                    #convert our variable into strings so that we can use them to index
                    j = str(okay)
                    #draw the data and define it as its variable and company
                    data[x+'.'+j] = data[(j)]
                    #append this data to the corresponding list 
                    data_list[j].append(data[x+'.'+j])
            else:
                #we print that there was no data for this day. This is common. The stock market
                #is closed on weekends and holidays.
                print('no data for day', t)
                #keep track of the days that we have missed
                t_values.append(t)
        #we have finished drawing data. now we want to save it
        print('finished pulling data on', x) 
        #We save each of the variables on each company
        for i in MEASUREMENTS:
            #np.unique(dates)
            j =str(i)
            #select the correct variable corresponding to our company
            data = data_list[j]
            #convert it to a pandas dataframe. Pass in the index that we store from earlier
            data = pd.DataFrame(data, index = dates)
            #save this file to a csv
            data.to_csv(x+'.'+j+'.csv')
        #find how many days did not have data collected on them   
        missed_days = len(t_values)
        #find how many days did have data collected on them
        made_days = len(dates)
        all_days = missed_days + made_days
        if all_days != TOTAL:
            print('It looks like you missed some days')
        else:
            print('Finished saving data on', x, 'during the', all_days, 'peroid there were',
                  missed_days, 'days missed and', made_days, 'days')
t_values = []
dates = []    
wut_iex(MEASUREMENTS)

Obtaining data for AES for day 1
Obtaining data for AES for day 2
no data for day 1
Obtaining data for AES for day 3
no data for day 2
Obtaining data for AES for day 4
Obtaining data for AES for day 5
Obtaining data for AES for day 6
Obtaining data for AES for day 7
Obtaining data for AES for day 8
Obtaining data for AES for day 9
no data for day 8
Obtaining data for AES for day 10
no data for day 9
Obtaining data for AES for day 11
Obtaining data for AES for day 12
Obtaining data for AES for day 13
Obtaining data for AES for day 14
Obtaining data for AES for day 15
Obtaining data for AES for day 16
no data for day 15
Obtaining data for AES for day 17
no data for day 16
Obtaining data for AES for day 18
Obtaining data for AES for day 19
Obtaining data for AES for day 20
Obtaining data for AES for day 21
Obtaining data for AES for day 22
Obtaining data for AES for day 23
no data for day 22
Obtaining data for AES for day 24
no data for day 23
Obtaining data for AES for day 25
Obtaining d

no data for day 65
Obtaining data for AEE for day 67
Obtaining data for AEE for day 68
finished pulling data on AEE
Finished saving data on AEE during the 68 there were 21 days missed and 47 days
Obtaining data for AEP for day 1
Obtaining data for AEP for day 2
no data for day 1
Obtaining data for AEP for day 3
no data for day 2
Obtaining data for AEP for day 4
Obtaining data for AEP for day 5
Obtaining data for AEP for day 6
Obtaining data for AEP for day 7
Obtaining data for AEP for day 8
Obtaining data for AEP for day 9
no data for day 8
Obtaining data for AEP for day 10
no data for day 9
Obtaining data for AEP for day 11
Obtaining data for AEP for day 12
Obtaining data for AEP for day 13
Obtaining data for AEP for day 14
Obtaining data for AEP for day 15
Obtaining data for AEP for day 16
no data for day 15
Obtaining data for AEP for day 17
no data for day 16
Obtaining data for AEP for day 18
Obtaining data for AEP for day 19
Obtaining data for AEP for day 20
Obtaining data for AEP 

Obtaining data for CNP for day 61
Obtaining data for CNP for day 62
Obtaining data for CNP for day 63
Obtaining data for CNP for day 64
Obtaining data for CNP for day 65
no data for day 64
Obtaining data for CNP for day 66
no data for day 65
Obtaining data for CNP for day 67
Obtaining data for CNP for day 68
finished pulling data on CNP
Finished saving data on CNP during the 68 there were 22 days missed and 46 days
Obtaining data for CMS for day 1
Obtaining data for CMS for day 2
no data for day 1
Obtaining data for CMS for day 3
no data for day 2
Obtaining data for CMS for day 4
Obtaining data for CMS for day 5
Obtaining data for CMS for day 6
Obtaining data for CMS for day 7
Obtaining data for CMS for day 8
Obtaining data for CMS for day 9
no data for day 8
Obtaining data for CMS for day 10
no data for day 9
Obtaining data for CMS for day 11
Obtaining data for CMS for day 12
Obtaining data for CMS for day 13
Obtaining data for CMS for day 14
Obtaining data for CMS for day 15
Obtainin

Obtaining data for D for day 61
Obtaining data for D for day 62
Obtaining data for D for day 63
Obtaining data for D for day 64
Obtaining data for D for day 65
no data for day 64
Obtaining data for D for day 66
no data for day 65
Obtaining data for D for day 67
Obtaining data for D for day 68
finished pulling data on D
Finished saving data on D during the 68 there were 21 days missed and 47 days
Obtaining data for DTE for day 1
Obtaining data for DTE for day 2
no data for day 1
Obtaining data for DTE for day 3
no data for day 2
Obtaining data for DTE for day 4
Obtaining data for DTE for day 5
Obtaining data for DTE for day 6
Obtaining data for DTE for day 7
Obtaining data for DTE for day 8
Obtaining data for DTE for day 9
no data for day 8
Obtaining data for DTE for day 10
no data for day 9
Obtaining data for DTE for day 11
Obtaining data for DTE for day 12
Obtaining data for DTE for day 13
Obtaining data for DTE for day 14
Obtaining data for DTE for day 15
Obtaining data for DTE for d

Obtaining data for EIX for day 56
Obtaining data for EIX for day 57
Obtaining data for EIX for day 58
no data for day 57
Obtaining data for EIX for day 59
no data for day 58
Obtaining data for EIX for day 60
Obtaining data for EIX for day 61
Obtaining data for EIX for day 62
Obtaining data for EIX for day 63
Obtaining data for EIX for day 64
Obtaining data for EIX for day 65
no data for day 64
Obtaining data for EIX for day 66
no data for day 65
Obtaining data for EIX for day 67
Obtaining data for EIX for day 68
finished pulling data on EIX
Finished saving data on EIX during the 68 there were 21 days missed and 47 days
Obtaining data for ETR for day 1
Obtaining data for ETR for day 2
no data for day 1
Obtaining data for ETR for day 3
no data for day 2
Obtaining data for ETR for day 4
Obtaining data for ETR for day 5
Obtaining data for ETR for day 6
Obtaining data for ETR for day 7
Obtaining data for ETR for day 8
Obtaining data for ETR for day 9
no data for day 8
Obtaining data for ETR

...

Obtaining data for XEL for day 35
Obtaining data for XEL for day 36
Obtaining data for XEL for day 37
no data for day 36
Obtaining data for XEL for day 38
no data for day 37
Obtaining data for XEL for day 39
Obtaining data for XEL for day 40
Obtaining data for XEL for day 41
Obtaining data for XEL for day 42
Obtaining data for XEL for day 43
Obtaining data for XEL for day 44
no data for day 43
Obtaining data for XEL for day 45
no data for day 44
Obtaining data for XEL for day 46
Obtaining data for XEL for day 47
Obtaining data for XEL for day 48
Obtaining data for XEL for day 49
Obtaining data for XEL for day 50
no data for day 49
Obtaining data for XEL for day 51
no data for day 50
Obtaining data for XEL for day 52
no data for day 51
Obtaining data for XEL for day 53
Obtaining data for XEL for day 54
Obtaining data for XEL for day 55
Obtaining data for XEL for day 56
Obtaining data for XEL for day 57
Obtaining data for XEL for day 58
no data for day 57
Obtaining data for XEL for day 5

Obtaining data for BHGE for day 29
Obtaining data for BHGE for day 30
no data for day 29
Obtaining data for BHGE for day 31
no data for day 30
Obtaining data for BHGE for day 32
Obtaining data for BHGE for day 33
Obtaining data for BHGE for day 34
Obtaining data for BHGE for day 35
Obtaining data for BHGE for day 36
Obtaining data for BHGE for day 37
no data for day 36
Obtaining data for BHGE for day 38
no data for day 37
Obtaining data for BHGE for day 39
Obtaining data for BHGE for day 40
Obtaining data for BHGE for day 41
Obtaining data for BHGE for day 42
Obtaining data for BHGE for day 43
Obtaining data for BHGE for day 44
no data for day 43
Obtaining data for BHGE for day 45
no data for day 44
Obtaining data for BHGE for day 46
Obtaining data for BHGE for day 47
Obtaining data for BHGE for day 48
Obtaining data for BHGE for day 49
Obtaining data for BHGE for day 50
no data for day 49
Obtaining data for BHGE for day 51
no data for day 50
Obtaining data for BHGE for day 52
no data 

no data for day 22
Obtaining data for XEC for day 24
no data for day 23
Obtaining data for XEC for day 25
Obtaining data for XEC for day 26
Obtaining data for XEC for day 27
Obtaining data for XEC for day 28
Obtaining data for XEC for day 29
Obtaining data for XEC for day 30
no data for day 29
Obtaining data for XEC for day 31
no data for day 30
Obtaining data for XEC for day 32
Obtaining data for XEC for day 33
Obtaining data for XEC for day 34
Obtaining data for XEC for day 35
Obtaining data for XEC for day 36
Obtaining data for XEC for day 37
no data for day 36
Obtaining data for XEC for day 38
no data for day 37
Obtaining data for XEC for day 39
Obtaining data for XEC for day 40
Obtaining data for XEC for day 41
Obtaining data for XEC for day 42
Obtaining data for XEC for day 43
Obtaining data for XEC for day 44
no data for day 43
Obtaining data for XEC for day 45
no data for day 44
Obtaining data for XEC for day 46
Obtaining data for XEC for day 47
Obtaining data for XEC for day 4

no data for day 16
Obtaining data for DVN for day 18
Obtaining data for DVN for day 19
Obtaining data for DVN for day 20
Obtaining data for DVN for day 21
Obtaining data for DVN for day 22
Obtaining data for DVN for day 23
no data for day 22
Obtaining data for DVN for day 24
no data for day 23
Obtaining data for DVN for day 25
Obtaining data for DVN for day 26
Obtaining data for DVN for day 27
Obtaining data for DVN for day 28
Obtaining data for DVN for day 29
Obtaining data for DVN for day 30
no data for day 29
Obtaining data for DVN for day 31
no data for day 30
Obtaining data for DVN for day 32
Obtaining data for DVN for day 33
Obtaining data for DVN for day 34
Obtaining data for DVN for day 35
Obtaining data for DVN for day 36
Obtaining data for DVN for day 37
no data for day 36
Obtaining data for DVN for day 38
no data for day 37
Obtaining data for DVN for day 39
Obtaining data for DVN for day 40
Obtaining data for DVN for day 41
Obtaining data for DVN for day 42
Obtaining data fo

Obtaining data for XOM for day 12
Obtaining data for XOM for day 13
Obtaining data for XOM for day 14
Obtaining data for XOM for day 15
Obtaining data for XOM for day 16
no data for day 15
Obtaining data for XOM for day 17
no data for day 16
Obtaining data for XOM for day 18
Obtaining data for XOM for day 19
Obtaining data for XOM for day 20
Obtaining data for XOM for day 21
Obtaining data for XOM for day 22
Obtaining data for XOM for day 23
no data for day 22
Obtaining data for XOM for day 24
no data for day 23
Obtaining data for XOM for day 25
Obtaining data for XOM for day 26
Obtaining data for XOM for day 27
Obtaining data for XOM for day 28
Obtaining data for XOM for day 29
Obtaining data for XOM for day 30
no data for day 29
Obtaining data for XOM for day 31
no data for day 30
Obtaining data for XOM for day 32
Obtaining data for XOM for day 33
Obtaining data for XOM for day 34
Obtaining data for XOM for day 35
Obtaining data for XOM for day 36
Obtaining data for XOM for day 37
no